# Aggregate anomaly labels from top-5 solutions from the ASHRAE competition

In [1]:
import pandas as pd

#### 1. Process the outlier lables from first ranked solution 
- The file rank1_bad_meter_readings.csv contains the outliers (is_bad_meter_reading == 1) 

In [2]:
rank1 = pd.read_csv('data/rank1_bad_meter_readings.csv')
print(rank1.info())
rank1.columns = ['rank1_outlier']
rank1.rank1_outlier.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 1 columns):
is_bad_meter_reading    int64
dtypes: int64(1)
memory usage: 154.2 MB
None


0    18901626
1     1314474
Name: rank1_outlier, dtype: int64

#### 2. Process the outlier lables from the second ranked solution 
- The file rank2_rows_to_drop.csv contains the index to the outlier rows

In [3]:
rank2_drop = pd.read_csv('data/rank2_rows_to_drop.csv', header=None, names = ['rank2_outlier'])
print(rank2_drop.info())

rank2 = rank1.copy()
rank2.columns = ['rank2_outlier']
rank2.rank2_outlier = 0
rank2.iloc[rank2_drop.rank2_outlier] = 1
rank2.rank2_outlier.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986260 entries, 0 to 986259
Data columns (total 1 columns):
rank2_outlier    986260 non-null int64
dtypes: int64(1)
memory usage: 7.5 MB
None


0    19229840
1      986260
Name: rank2_outlier, dtype: int64

#### 3. Process the outlier lables from third ranked solution 
- The file rank3_rows_to_drop.csv contains the index to the outlier rows

In [4]:
# train1 = pd.read_feather('./data/rank3_train_cleanup_001.feather')
# print(train1.info())
rank3_drop = pd.read_csv('./data/rank3_rows_to_drop.csv', header=None, names = ['rank3_outlier'])
print(rank3_drop.info())

rank3 = rank1.copy()
rank3.columns = ['rank3_outlier']
rank3.rank3_outlier = 0
rank3.iloc[rank3_drop.rank3_outlier] = 1
rank3.rank3_outlier.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525023 entries, 0 to 525022
Data columns (total 1 columns):
rank3_outlier    525023 non-null int64
dtypes: int64(1)
memory usage: 4.0 MB
None


0    19691077
1      525023
Name: rank3_outlier, dtype: int64

#### 4. Process the outlier lables from fourth ranked solution 

In [5]:
# # https://github.com/buds-lab/ashrae-great-energy-predictor-3-solution-analysis/blob/master/solutions/rank-4/exception_label/fork-of-ashrae-eda-exception-label5.ipynb
# rank4_exp = pd.read_pickle('./data/rank4_train_exception.pkl')
# rank4_exp.to_csv('./data/rank4_train_exception.csv', index=False)
# print(rank4_exp.info())
# rank4_exp.exception.value_counts()

In [6]:
# rank4_train = pd.read_pickle('./data/rank4_train_df.pkl')
# #print(rank4_train.info())
# rank4_train.exception.value_counts()

##### Outliers from model1

In [7]:
# # https://github.com/buds-lab/ashrae-great-energy-predictor-3-solution-analysis/blob/master/solutions/rank-4/model1/fork-of-as-2kfold-model6-xgb-fr7d12-fold0.ipynb
# tr_x = rank4_train.copy()
# tr_x = tr_x[(((tr_x.site_id == 0) & (tr_x.exception != 1) & (tr_x.exception != 3)) | (tr_x.site_id != 0)) & (tr_x.exception != 4)]
# print(rank4_train.shape, tr_x.shape, rank4_train.shape[0]-tr_x.shape[0])
# print(tr_x.exception.value_counts())

# outlier_rows = rank4_train.index.difference(tr_x.index)
# rows_to_drop = pd.DataFrame({'rows_to_drop':outlier_rows})
# rows_to_drop.to_csv('./data/rank4_model1_rows_to_drop.csv', index=False)
# rows_to_drop.info()

##### Outliers from model2

In [8]:
# #https://github.com/buds-lab/ashrae-great-energy-predictor-3-solution-analysis/blob/master/solutions/rank-4/model2/as-meter2-no-1099-xgb-meter0-fold0.ipynb
# tr_x = rank4_train.copy()    
# tr_x = tr_x[(tr_x.exception != 3) & (tr_x.exception != 1) & (tr_x.exception != 4)]
# print(rank4_train.shape, tr_x.shape, rank4_train.shape[0]-tr_x.shape[0])
# print(tr_x.exception.value_counts())

# outlier_rows = rank4_train.index.difference(tr_x.index)
# rows_to_drop = pd.DataFrame({'rows_to_drop':outlier_rows})
# rows_to_drop.to_csv('./data/rank4_model2_rows_to_drop.csv', index=False)
# rows_to_drop.info()

##### Outliers from model3

In [9]:
# # https://github.com/buds-lab/ashrae-great-energy-predictor-3-solution-analysis/blob/master/solutions/rank-4/model3/ashrae-kfold-lightgbm-without-leak-1-08.ipynb
# train_filtered = rank4_train.copy()
# train_filtered = train_filtered[train_filtered['building_id'] != 1099]
# train_filtered = train_filtered.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

# print(rank4_train.shape, train_filtered.shape, rank4_train.shape[0] - train_filtered.shape[0])

# outlier_rows = rank4_train.index.difference(train_filtered.index)
# rows_to_drop = pd.DataFrame({'rows_to_drop':outlier_rows})
# rows_to_drop.to_csv('./data/rank4_model3_rows_to_drop.csv', index=False)
# rows_to_drop.info()

In [10]:
rank4_m1 = pd.read_csv('./data/rank4_model1_rows_to_drop.csv')
rank4_m2 = pd.read_csv('./data/rank4_model2_rows_to_drop.csv')
rank4_m3 = pd.read_csv('./data/rank4_model3_rows_to_drop.csv')

rank4 = rank1.copy()
rank4.columns = ['model1']
rank4.model1 = 0
rank4['model2'] = rank4.model1
rank4['model3'] = rank4.model1

rank4.loc[rank4_m1.rows_to_drop, 'model1'] = 1
rank4.loc[rank4_m2.rows_to_drop, 'model2'] = 1
rank4.loc[rank4_m3.rows_to_drop, 'model3'] = 1
rank4['outlier'] = rank4.max(axis=1)  # aggregate outliers from all models

rank4.to_csv('./data/rank4_rows_to_drop.csv', index=False)

print(rank4.apply(pd.Series.value_counts))
rank4.describe().T

     model1    model2    model3   outlier
0  19857254  19395215  19852320  19379447
1    358846    820885    363780    836653


,count,mean,std,min,25%,50%,75%,max
model1,20216100.0,0.017751,0.132043,0.0,0.0,0.0,0.0,1.0
model2,20216100.0,0.040606,0.197375,0.0,0.0,0.0,0.0,1.0
model3,20216100.0,0.017995,0.132931,0.0,0.0,0.0,0.0,1.0
outlier,20216100.0,0.041385,0.199180,0.0,0.0,0.0,0.0,1.0


In [11]:
rank4 = pd.read_csv('./data/rank4_rows_to_drop.csv')
print(rank4.info())
print(rank4.describe().T)

rank4 = rank4[['outlier']]
rank4.columns = ['rank4_outlier']
rank4.rank4_outlier.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
model1     int64
model2     int64
model3     int64
outlier    int64
dtypes: int64(4)
memory usage: 616.9 MB
None
              count      mean       std  min  25%  50%  75%  max
model1   20216100.0  0.017751  0.132043  0.0  0.0  0.0  0.0  1.0
model2   20216100.0  0.040606  0.197375  0.0  0.0  0.0  0.0  1.0
model3   20216100.0  0.017995  0.132931  0.0  0.0  0.0  0.0  1.0
outlier  20216100.0  0.041385  0.199180  0.0  0.0  0.0  0.0  1.0


0    19379447
1      836653
Name: rank4_outlier, dtype: int64

#### 5. Process the outlier lables from fifth ranked solution 

In [12]:
# # Note: run this once
# #https://github.com/buds-lab/ashrae-great-energy-predictor-3-solution-analysis/blob/master/solutions/rank-5/preproceeding_code/prepare_data.py

# import numpy as np
# import pandas as pd
# import argparse
# from copy import deepcopy
# from datetime import date, datetime, timedelta
# from tqdm import tqdm
# import os 

# train = pd.read_csv('./data/train.csv', parse_dates=['timestamp'])
# # test = pd.read_csv(f'{code_path}/data/{args.test_file}', parse_dates=['timestamp'])
# # building_meta = pd.read_csv(f'{code_path}/data/building_metadata.csv')
# # weather_train = pd.read_csv(f'{code_path}/data/{args.weather_train_file}', parse_dates=['timestamp'])
# # weather_test = pd.read_csv(f'{code_path}/data/{args.weather_test_file}', parse_dates=['timestamp'])

# # drop bad rows

# del_list = list()

# for building_id in range(1449):
#     train_gb = train[train['building_id'] == building_id].groupby("meter")

#     for meter, tmp_df in train_gb:
# #         print("building_id: {}, meter: {}".format(building_id, meter))
#         data = tmp_df['meter_reading'].values
# #         splited_value = np.split(data, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
# #         splited_date = np.split(tmp_df.timestamp.values, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
#         splited_idx = np.split(tmp_df.index.values, np.where((data[1:] != data[:-1]) | (data[1:] == min(data)))[0] + 1)
#         for i, x in enumerate(splited_idx):
#             if len(x) > 24:
# #                 print("length: {},\t{}-{},\tvalue: {}".format(len(x), x[0], x[-1], splited_value[i][0]))
#                 del_list.extend(x[1:])
                
                
# #         print()

# del tmp_df, train_gb


# def idx_to_drop(df):
#     drop_cols = []
#     electric_zero = df[(df['meter']==0)&(df['meter_reading']==0)].index.values.tolist()
#     drop_cols.extend(electric_zero)
#     not_summer = df[(df['timestamp'].dt.month!=7)&(df['timestamp'].dt.month!=8)]
#     not_summer['cumsum'] = not_summer.groupby(['building_id','meter'])['meter_reading'].cumsum()
#     not_summer['shifted'] = not_summer.groupby(['building_id','meter'])['cumsum'].shift(48)
#     not_summer['difference'] = not_summer['cumsum']-not_summer['shifted']
#     steam_zero = not_summer[(not_summer['difference']==0) & (not_summer['meter']==2)].index.values.tolist()
#     hotwater_zero = not_summer[(not_summer['difference']==0) & (not_summer['meter']==3)].index.values.tolist()
#     drop_cols.extend(steam_zero)
#     drop_cols.extend(hotwater_zero)
#     del not_summer
#     not_winter = train[(df['timestamp'].dt.month!=12)&(df['timestamp'].dt.month!=1)]
#     not_winter['cumsum'] = not_winter.groupby(['building_id','meter'])['meter_reading'].cumsum()
#     not_winter['shifted'] = not_winter.groupby(['building_id','meter'])['cumsum'].shift(48)
#     not_winter['difference'] = not_winter['cumsum']-not_winter['shifted']
#     chilled_zero = not_winter[(not_winter['difference']==0) & (not_winter['meter']==1)].index.values.tolist()
#     drop_cols.extend(chilled_zero)
#     return drop_cols

# del_list.extend(idx_to_drop(train))



# del_list_new = train.loc[del_list].index#query('timestamp < 20160901').index

# train = train.drop(del_list_new)

# train = train.query('(not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")) & (not (meter==0 & meter_reading==0))')

# train.info()

In [13]:
# train_ = pd.read_csv('./data/train.csv', parse_dates=['timestamp'])
# outlier_rows = train_.index.difference(train.index)

# rows_to_drop = pd.DataFrame({'rows_to_drop':outlier_rows})
# rows_to_drop.to_csv('./data/rank5_rows_to_drop.csv', index=False)
# rows_to_drop.info()

In [14]:
rank5_outliers = pd.read_csv('./data/rank5_rows_to_drop.csv')

rank5 = rank1.copy()
rank5.columns = ['rank5_outlier']
rank5.rank5_outlier = 0
rank5.iloc[rank5_outliers.rows_to_drop] = 1
rank5.rank5_outlier.value_counts()

0    18887522
1     1328578
Name: rank5_outlier, dtype: int64

#### Merge all anomaly labels

In [32]:
rank4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 1 columns):
rank4_outlier    int64
dtypes: int64(1)
memory usage: 154.2 MB


In [34]:
df_anomaly = pd.concat([rank1, rank2, rank3, rank5], axis=1, sort=False)
df_anomaly['outlier'] = df_anomaly.max(axis=1)  # aggregated outliers from all solutions

print(df_anomaly.info())
df_anomaly.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 5 columns):
rank1_outlier    int64
rank2_outlier    int64
rank3_outlier    int64
rank5_outlier    int64
outlier          int64
dtypes: int64(5)
memory usage: 771.2 MB
None


,count,mean,std,min,25%,50%,75%,max
rank1_outlier,20216100.0,0.065021,0.246563,0.0,0.0,0.0,0.0,1.0
rank2_outlier,20216100.0,0.048786,0.215420,0.0,0.0,0.0,0.0,1.0
rank3_outlier,20216100.0,0.025971,0.159047,0.0,0.0,0.0,0.0,1.0
rank5_outlier,20216100.0,0.065719,0.247790,0.0,0.0,0.0,0.0,1.0
outlier,20216100.0,0.104979,0.306526,0.0,0.0,0.0,0.0,1.0


In [36]:
df_anomaly.to_csv('./data/anomaly_labels_all.csv', index=False)
df_anomaly[['outlier']].to_csv('./data/anomaly_labels.csv', index=False)